#### Import and hugging face initial setup


In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 97.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
!pip -q install transformers
!pip -q install sentencepiece
!pip -q install accelerate
!pip install -q bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 10.1 MB/s eta 0:00:00


In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer,AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import BitsAndBytesConfig

#### Load dataset

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00


In [ ]:
# Load the MATH dataset from Hugging Face
from datasets import load_dataset

# math_dataset = load_dataset('math_dataset', 'algebra__linear_1d')
math_dataset_arith=load_dataset('math_dataset','arithmetic__add_or_sub')

Generating train split:   0%|          | 0/1999998 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

#### Local data

In [ ]:
import json

# Specify the path to your JSON file
file_path = '/content/data.json'

# Open the file and load the JSON data
with open(file_path, 'r') as json_file:
    math_dataset_arith = json.load(json_file)



In [ ]:
# Assuming math_dataset_arith is your dictionary
math_dataset_arith = {k: math_dataset_arith[k] for k in list(math_dataset_arith)[:50]}


In [ ]:
math_dataset_arith["question"] = math_dataset_arith["question"][:50]
math_dataset_arith["answer"] = math_dataset_arith["answer"][:50]

#### Gemma setup

In [ ]:
class AgentManager:
    def __init__(self):
        self.agents = {}

    def create_agent(self, model_name, nick_name):
        config = BitsAndBytesConfig(load_in_8bit=True)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name,device_map = "auto",config)
        self.agents[nick_name] = {'model': model, 'tokenizer': tokenizer}
        print(f"Agent '{nick_name}' created successfully.")

    def interact_with_agent(self, agent_name, prompt):
        # if agent_name not in self.agents:
        #     print(f"Agent '{agent_name}' not found.")
        #     return None
        print("Agent present")
        agent = self.agents[agent_name]
        input_ids = agent['tokenizer'](prompt, return_tensors="pt").to('cuda' if torch.cuda.is_available() else 'cpu')
        outputs = agent['model'].generate(**input_ids, max_length=2048)
        # decode(outputs[0])
        return agent['tokenizer'].decode(outputs[0])

In [ ]:
def get_gemma_answer(response_gemma, question):
  return response_gemma.replace(question,"").replace("<bos>","").replace("<eos>","").replace("\n","")

In [ ]:
agent = AgentManager()

# Create an agent named "gemma"
agent.create_agent("google/gemma-2b-it",'gemma')

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Agent 'gemma' created successfully.


#### Orca setup

In [ ]:
# Hugging Face model_path load model

model_path_orca = 'psmathur/orca_mini_3b'
# config = BitsAndBytesConfig(load_in_8bit=True)
tokenizer_orca = LlamaTokenizer.from_pretrained(model_path_orca)
model_orca = LlamaForCausalLM.from_pretrained(model_path_orca, torch_dtype=torch.float16, device_map='auto')


# generate text function
def generate_text_orca(system, instruction, input=None):

    if input:
        prompt = f"### System:\n{system}\n\n### User:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
    else:
        prompt = f"### System:\n{system}\n\n### User:\n{instruction}\n\n### Response:\n"

    tokens = tokenizer_orca.encode(prompt)
    tokens = torch.LongTensor(tokens).unsqueeze(0)
    tokens = tokens.to('cuda')

    instance = {'input_ids': tokens,'top_p': 1.0, 'temperature':0.7, 'generate_len': 1024, 'top_k': 50}

    length = len(tokens[0])
    with torch.no_grad():
        rest = model_orca.generate(
            input_ids=tokens,
            max_length=length+instance['generate_len'],
            use_cache=True,
            do_sample=True,
            top_p=instance['top_p'],
            temperature=instance['temperature'],
            top_k=instance['top_k']
        )
    output = rest[0][length:]
    string = tokenizer_orca.decode(output, skip_special_tokens=True)
    return f'[!] Response: {string}'

tokenizer_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/534k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/3.72G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
# MAD Workflow for a list of questions:
def summarize_responses_modified(response1, response2, q1):
    summarized_response = str(q1) + "  This is gemma agent's answer for the question: " + response1 + " This is orca agent's answer for same question - " + response2 + " Now, Recheck those two answers and give the correct answer?."
    return summarized_response
import re

rounds = 2
system_orca = 'You are an AI assistant that follows instruction extremely well. Help as much as you can.' # For orca initial setup
final_answers_generated = [] # Final answer list to check accuracy

for question in math_dataset_arith["train"]['question'][0:50]:
  question = question.split("'")[1].rstrip("\\n")
  responses = []
  summarized_response = question

  for i in range(rounds):
      # Gemma
      response_gemma = agent.interact_with_agent('gemma', summarized_response)
      response_gemma = get_gemma_answer(response_gemma,summarized_response)

      # Orca
      response_orca = generate_text_orca(system_orca, summarized_response)
      response_orca = re.sub(r"\n+", "\n", response_orca.replace("[!]", "").strip()).replace("Response:", "") #remove extra lines and [!]


      # Concatenate response
      summarized_response = summarize_responses_modified(response_gemma, response_orca, question).replace("Now, Recheck those two answers and give the correct answer?","")
      # print(summarized_response)

      responses.append(summarized_response)

  ''' After running for specified rounds, get one final answer from orca stored in the last summary we get after the last round.
      We can also choose any other model here in place of orca.
      Also, we can use summarized_response directly instead of storing responses for optimization, need to think '''

  final = generate_text_orca(system_orca, responses[-1]+" Give only final answer")
  final_numerical_answers = re.findall(r"[-+]?\d[\d,]*(?:\.\d+)?", final) # Get all numerical values from the final
  final_answer = final_numerical_answers[-1] if final_numerical_answers else "No answer found" # Selecting the last numerical value as the final answer

  final_answers_generated.append(final_answer) # add final answer for this question to the list

Agent present
Agent present
Agent present
Agent present


In [ ]:
# Actual true answers

answers_true = [answer.split("'")[1].rstrip("\\n") for answer in math_dataset_arith['train']['answer'][0:2]]
answers_true[0:2]

['-1977321734.94', '436273717']

In [ ]:
# Calculate accuracy

num_correct = 0
for generated_answer, true_answer in zip(final_answers_generated, answers_true):
    if generated_answer == true_answer:
        num_correct += 1

total_questions = len(answers_true)
accuracy = num_correct / total_questions * 100

print("Accuracy:", accuracy, "%")
print("Num of correct:", num_correct)